In [19]:
import yaml 
import glob 
import pickle
import pandas as pd
pd.options.display.precision = 2
files = glob.glob("*.yaml")
print(f"実験結果数 : {len(files)}")

path = f'../../../data/Hazumi_features/Hazumi1911_features.pkl'
_, SS_binary, SS_ternary, _, TS_binary, TS_ternary, SP, TP, Text, Audio, Visual, vid = pickle.load(open(path, 'rb'), encoding='utf-8')

columns = ['E', 'A', 'C', 'N', 'O']
df = pd.DataFrame.from_dict(TP, orient='index', columns=columns)
tp_df = (df - df.mean() ) / df.std(ddof=0)

def profiling(profile, id, TP=None):
    age = int(id[5])
    gender = id[4]

    per_th = 0

    if profile == 0:
        res = 0
    elif profile == 1:
        if gender == 'F':
            res = 0
        else:
            res = 1
    elif profile == 2:
        if age <= 4:
            res = 0 
        else:
            res = 1
    elif profile == 3:
        if gender == 'F':
            if age <= 4:
                res = 0
            else:
                res = 1
        else:
            if age <= 4:
                res = 2 
            else:
                res = 3
    elif profile == 4:
        res = 1 if TP.loc[id, :][0] >= per_th else 0
    elif profile == 5:
        res = 1 if TP.loc[id, :][1] >= per_th else 0
    elif profile == 6:
        res = 1 if TP.loc[id, :][2] >= per_th else 0
    elif profile == 7:
        res = 1 if TP.loc[id, :][3] >= per_th else 0
    elif profile == 8:
        res = 1 if TP.loc[id, :][4] >= per_th else 0
    return res


実験結果数 : 63


# 全体的な結果

In [20]:
index = [0, 1, 2, 3, 4, 5, 6, 7, 8]
df_results = pd.DataFrame(0, index=index, columns=['t','a','v','ta','tv','av','tav'])
for file in files:
    with open(file) as file:
        obj = yaml.safe_load(file)
        config = obj["config"]
        if config["annot"] == "SS" and config["label"] == "ternary":
            results = obj["results"]
            data = list(results.values())
            acc = round(sum(data) / len(data), 3)
            df_results.at[config['pmode'], config['modal']] = acc

display(df_results)

,t,a,v,ta,tv,av,tav
0,0.49,0.47,0.46,0.48,0.50,0.48,0.48
1,0.49,0.44,0.42,0.45,0.47,0.43,0.45
2,0.49,0.49,0.52,0.47,0.49,0.48,0.47
3,0.54,0.47,0.52,0.48,0.52,0.48,0.48
4,0.46,0.48,0.50,0.46,0.46,0.48,0.47
5,0.48,0.43,0.43,0.44,0.48,0.43,0.44
6,0.49,0.44,0.43,0.44,0.48,0.44,0.43
7,0.48,0.45,0.44,0.44,0.49,0.46,0.45
8,0.48,0.43,0.44,0.44,0.49,0.44,0.45


# ユーザ特性ごとの実験結果

In [33]:
def show_group_results(pmode=1, group_labels=None):
    index = [0, pmode]
    for i in range(len(group_labels)):
        df_results = pd.DataFrame(0, index=index, columns=['t','a','v','ta','tv','av','tav'])
        for file in files:
            with open(file) as file:
                obj = yaml.safe_load(file)
                config = obj["config"]
                if config["pmode"] == 0 or config["pmode"] == pmode:
                    results = []
                    for key, data in obj["results"].items():
                        if profiling(pmode, key, tp_df) == i:
                            results.append(data)
                    acc = round(sum(results) / len(results), 2)
                    df_results.at[config['pmode'], config['modal']] = acc
        print(group_labels[i])
        display(df_results)

In [36]:
# 性別
print("性別")
pmode = 1
group_labels = ["女性", "男性"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 年齢
print("年齢")
pmode = 2
group_labels = ["50歳未満", "50歳以上"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 性別&年齢
print("性別&年齢")
pmode = 3
group_labels = ["女性&50歳未満", "女性&50歳以上", "男性&50歳未満", "男性&50歳以上"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 外向性
print("外向性")
pmode = 4
group_labels = ["Low", "High"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 協調性
print("協調性")
pmode = 5
group_labels = ["Low", "High"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 勤勉性
print("勤勉性")
pmode = 6
group_labels = ["Low", "High"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 神経症傾向
print("神経症傾向")
pmode = 7
group_labels = ["Low", "High"]
show_group_results(pmode=pmode, group_labels=group_labels)

# 開放性
print("開放性")
pmode = 8
group_labels = ["Low", "High"]
show_group_results(pmode=pmode, group_labels=group_labels)


性別
女性


,t,a,v,ta,tv,av,tav
0,0.40,0.40,0.41,0.41,0.42,0.41,0.42
1,0.39,0.34,0.34,0.36,0.37,0.33,0.37


男性


,t,a,v,ta,tv,av,tav
0,0.59,0.56,0.53,0.55,0.59,0.56,0.56
1,0.61,0.56,0.53,0.55,0.60,0.55,0.54


年齢
50歳未満


...

,t,a,v,ta,tv,av,tav
0,0.41,0.42,0.43,0.42,0.41,0.42,0.43
7,0.42,0.38,0.37,0.38,0.42,0.39,0.39


開放性
Low


,t,a,v,ta,tv,av,tav
0,0.43,0.42,0.41,0.43,0.45,0.42,0.44
8,0.40,0.35,0.40,0.40,0.43,0.36,0.39


High


,t,a,v,ta,tv,av,tav
0,0.54,0.52,0.51,0.52,0.54,0.52,0.52
8,0.54,0.50,0.48,0.47,0.54,0.50,0.50
